# [0] Imports

In [1529]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets, Dates

# [1] Set Up Parameters

In [1530]:
n_jobs = 25
m_cov = 25
B = n_jobs
V = m_cov
T = 2 * n_jobs
min_duration = 2
max_duration = 6
speed = 500 / 3
radius = 50
size = 500
mesh = 50;

# [2] Create Job Parameters

In [1531]:
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    min_jobs_allowed = n_jobs >= 15 ? 3 : 2
        target_clusters = 5
        while jobs_created != jobs_total
            num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
            jobs_created += num_to_add
            push!(cluster, num_to_add)
        end
    
    return cluster
end;

time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []
cluster = create_cluster_sizes(n_jobs)

function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end

time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)

function reformat_locations(locations)
    #= The fact that locations[1] gives the same float (not tuple or list)
    as locations[1, 1] and locations[1][2] throws an 
    error is extremely frustrating.
    This function makes locations[1][1] possible, 
    while locations[1] correctly returns a tuple. =#
    locs = []
    for i in 1:Int(length(locations) * 0.5)
        push!(locs, [locations[i,1], locations[i,2]])
    end
    return locs
end

job_locs = reformat_locations(locations)
push!(job_locs, job_locs[1]);

In [1532]:
job_dists = [LinearAlgebra.norm(job_locs[i] .- job_locs[j]) for i=1:n_jobs+2, j = 1:n_jobs+2];

In [1533]:
job_travel_times = ceil.(job_dists / speed);

# [3] Create Coverage Parameters

In [1534]:
create_all_valid_spots = true;

In [1535]:
function is_close_enough(coverage_loc, job_loc)
    #= Given a coverage_loc with 2 elements x y
    and a job_loc with two elements x y
    determines if that specific job_loc is located within r 
    of the coverage_loc as the crow flies =#
    return (coverage_loc[1]-job_loc[1])^2 + (coverage_loc[2]-job_loc[2])^2 <= radius^2
end;

function close_enough_jobs(coverage_loc)
    jobs = []
    for i in 1:n_jobs
        if is_close_enough(coverage_loc, job_locs[i+1])
            push!(jobs, i)
        end
    end
    return jobs
end;

In [1536]:
cov_locs = []
seen = []

for x in 0:mesh:size
    for y in 0:mesh:size
        potential_cov = [x, y]
        useful = false
        
        if create_all_valid_spots
            for index in 2:length(job_locs)-1
                if is_close_enough(potential_cov, job_locs[index])
                    useful = true
                    break
                end
            end
        else
            for job in close_enough_jobs(potential_cov)
                if ~(job in seen)
                    push!(seen, job)
                    useful = true
                end
            end
        end
        
        if useful
            push!(cov_locs, potential_cov)
        end
    end
end;

In [1537]:
m_cov = length(cov_locs);

In [1538]:
insert!(cov_locs, 1, job_locs[1])
push!(cov_locs, cov_locs[1]);

In [1539]:
cov_dists = [LinearAlgebra.norm(cov_locs[i] .- cov_locs[j]) for i=1:m_cov+2, j = 1:m_cov+2];

In [1540]:
cov_travel_times = ceil.(cov_dists / speed);

# [4] Create Initial Job Routes

In [1541]:
job_routes = []

for i in 1:n_jobs
    route = []
    dist = job_dists[0+1, i+1]
    
    start_going_time = Int(floor(time_windows[i][1] - dist/speed))
    # time_arrival = time_windows[i][1]
    # time_finished = time_windows[i][2]
    return_finished = Int(ceil(time_windows[i][2] + dist/speed))
    
    push!(route, [[0, start_going_time], [i, time_windows[i][1]], dist])
    
    for working in time_windows[i][1]:(time_windows[i][2]-1)
        push!(route, [[i, working], [i, working+1], 0])
    end
    
    push!(route, [[i, time_windows[i][2]], [n_jobs+1, return_finished], dist])
    push!(job_routes, route)
end;

# [5] Create Initial Coverage Routes

In [1542]:
cov_routes = []

for j in 1:m_cov
    route = []
    dist = cov_dists[0+1, j+1]
    
    arrive_time = Int(ceil(dist/speed))
    leave_time = Int(T+1 - ceil(dist/speed))
    
    push!(route, [[0, 0], [j, arrive_time], dist])

    for t in arrive_time:leave_time-1
        push!(route, [ [j, t], [j, t+1], 0])
    end
    
    push!(route, [[j, leave_time], [m_cov+1, T+1], dist])
    push!(cov_routes, route)
end;

# [6] Helper Functions

## [6.1] Compute cost of route

In [1543]:
function route_cost(route)
    ans = 0
    for arc in route
        ans += arc[3]
    end
    return ans
end;

## [6.2] Compute $u_i^q$

In [1544]:
function compute_u(routes)
    Q = length(routes)
    u = [[0 for q in 1:Q] for i in 1:n_jobs]
    
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1 = arc[1][1]
            loc2 = arc[2][1]
            if (1 <= loc1 <= n_jobs)
                u[loc1][rindex] = 1
            end
            if (1 <= loc2 <= n_jobs)
                u[loc2][rindex] = 1
            end
        end
    end
    return u
end;

## [6.3] Compute $\delta_{it}^q$

In [1545]:
function compute_delta(routes)
    Q = length(routes)
    delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1, time1 = arc[1]
            loc2, time2 = arc[2]
            if (loc1 != 0) & (loc1 != n_jobs + 1)
                delta[loc1][time1][rindex] = 1
            end
            
            if (loc2 != 0) & (loc2 != n_jobs + 1)
                delta[loc2][time2][rindex] = 1
            end
        end
    end
    return delta
end;

## [6.4] Compute $y_{jt}^p$

In [1546]:
function compute_y(routes)
    y = [[[0 for p in 1:length(routes)] for t in 1:T] for j in 1:m_cov]
    
    for rindex in 1:length(routes)
        route = routes[rindex]
        for arc in route
            job1, time1 = Int.(arc[1])
            job2, time2 = Int.(arc[2])
            if job1 != 0
                y[job1][time1][rindex] = 1
            end
            if job2 != m_cov + 1
                y[job2][time2][rindex] = 1
            end
        end
    end
    return y
end;

## [6.5] Compute $L_{ji}$

In [1547]:
function compute_L(cov_locs, job_locs)
    #= Given a list of tuple coverage locations and job locations,
    returns a matrix of L values. Specifically, 
    L[j][i], indicating if spot j is close enough to node i
    for coverage to happen. We might have to delete padding of depot
    first. =#
    L = []
    for j in 2:length(cov_locs)-1
        L_j = []
        cov_loc = cov_locs[j] # padding accounted for
        for i in 2:length(job_locs)-1
            push!(L_j, is_close_enough(cov_loc, job_locs[i]) ? 1 : 0)
        end
        push!(L, L_j)
    end
    
    return L
end;

## [6.6] Compute $j$'s Neighborhood

In [1548]:
function neighborhood(j)
    # returns all nodes i which are close to the spot j.
    cov_loc = cov_locs[j+1] # don't forget padding!
    
    close = []
    for i in 1:n_jobs
        if is_close_enough(cov_loc, job_locs[i+1])
            push!(close, i)
        end
    end
    
    return close
end;

## [6.7] Compute coverage stations close to $i$

In [1549]:
function close_covs(i)
    job_loc = job_locs[i+1] # padding included
    close = []
    for j in 1:m_cov
        if is_close_enough(cov_locs[j+1], job_loc)
            push!(close, j)
        end
    end
    return close
end;

## [6.8] Given route, list nodes

In [1550]:
function route_to_nodes(route)
    # get only the 1st node of all arcs except the last, when you get the 2nd
    nodes = []
    for arc in route[1:end-1]
        new_node = arc[1][1]
        push!(nodes, new_node)
    end
    push!(nodes, route[end][2][1])
    return unique(nodes)
end;

## [6.9] Given route, list times

In [1551]:
function route_to_times(route)
    #= Obtain time "markers" when you leave
    or arrive at any node. This is more convenient
    than just telling me when I arrive at any node.
    If the two nodes in an arc are not equal,
    list both of their times. Otherwise, list neither. =#
    times = []
    for arc in route[1:end]
        n1, t1 = arc[1]
        n2, t2 = arc[2]
        if n1 != n2
            push!(times, t1)
            push!(times, t2)
        end
    end
    return times
end;

## [6.10] Given nodes and times, produce route

In [1552]:
function nodes_and_times_to_route(nodes, times, route_type)
    #= Given by two components: When you leave/go between nodes,
    and when you stay. Alternate: start "when you stay",
    then "when you go". For the first node there is no stay. 
    For the last node there is no stay nor go.=#
    route = []
    for i in 1:length(nodes)-1
        
        # WHEN YOU STAY
        if i > 1
            n = nodes[i]
            t1 = times[2*i-2]
            t2 = times[2*i-1]
            
            for t in t1:t2-1
                arc = [[n, t], [n, t+1], 0]
                push!(route, arc)
            end
        end
        
        # WHEN YOU GO
        n1 = nodes[i]
        n2 = nodes[i+1]
        t1 = times[2*i-1]
        t2 = times[2*i]
        dist = route_type == "job" ? job_dists[n1+1, n2+1] : cov_dists[n1+1, n2+1]
        arc = [[n1, t1], [n2, t2], dist]
        push!(route, arc)
    end
    return route
end;       

# [7] Shortest Paths for Job Routes

In [1553]:
function job_sp(dists, travel_times, windows, loads, rho, pi, xi)
    paths, rcs, times = [[0]], [rho], [[0]]
    curp, endp = 1, 1
    
    pruner = [Inf for i in 1:n_jobs]
    
    while true
        curpath = paths[curp]
        curtimes = times[curp]
        if last(curpath) == n_jobs+1
            curp += 1
            if curp > endp
                break
            else
                continue
            end
        end
        
        for i in 1:n_jobs+1
            if ~(i in curpath)
                
                last_depart_time = last(curtimes)
                travel_time = travel_times[last(curpath)+1, i+1]
                new_job_load = (i < n_jobs+1) ? loads[i] : 0
                new_window_start, new_window_close = (i < n_jobs+1) ? windows[i] : [0, T+1]
                proposed_arrival = last_depart_time + travel_time + new_job_load
                if proposed_arrival > new_window_close
                    continue
                end
                
                new_node_arrival = max(proposed_arrival, new_window_start)
                new_node_depart = new_node_arrival + new_job_load
                cur_rc = rcs[curp]
                cur_rc -= (i < n_jobs + 1) ? pi[i] : 0;
                
                for t in new_node_arrival:new_node_depart
                    cur_rc += (i < n_jobs + 1) ? xi[i, Int(t)] : 0;
                end
                
                if i < n_jobs + 1 && cur_rc < pruner[i]
                    pruner[i] = cur_rc
                elseif i == n_jobs+1
                else
                    continue
                end
                
                push!(paths, push!(copy(curpath), i))
                push!(rcs, cur_rc)
                if i < n_jobs + 1
                    push!(times, push!(push!(copy(curtimes), new_node_arrival), new_node_depart))
                else
                    push!(times, push!(copy(curtimes), new_node_arrival))
                end
                
                endp += 1
            end
        end
        
        curp += 1
        
        if curp == endp
            break
        end
    end
    
    return paths, rcs, times
end;

# [8] Shortest Paths for Coverage Routes

In [1554]:
function all_jobs_ended(spot)
    last = -Inf
    for i in neighborhood(spot)
        if time_windows[i][2] > last
            last = time_windows[i][2]
        end
    end
    return last
end;

In [1555]:
function when_to_leave(spot)
    neigh = neighborhood(spot)
    leavable_times = []
    time_ended = all_jobs_ended(spot)
    for t in 1:T
        can_leave = true
        for i in neigh
            if time_windows[i][1] <= t <= time_windows[i][2] && time_windows[i][1] <= t+1 <= time_windows[i][2]
                can_leave = false
            end
        end
        
        if can_leave && t <= time_ended
            push!(leavable_times, t)
        end
    end
    return leavable_times
end;

In [1556]:
function cov_sp(dists, travel_times, beta, xi)
    paths, rcs, times = [[0]], [beta], [[0]]
    curp, endp = 1, 1
    last_job_ended = [all_jobs_ended(j) for j in 1:m_cov]
    
    pruner = [Inf for j in 1:m_cov]
    
    while true
        curpath = paths[curp]
        curtimes = times[curp]
        if last(curpath) == m_cov+1
            curp += 1
            if curp > endp
                break
            else
                continue
            end
        end
        
        for j in 1:m_cov+1
            #anything goes.
            prev_depart_time = last(curtimes)
            travel_time = travel_times[last(curpath)+1, j+1]
            new_spot_arrival = prev_depart_time + travel_time
            departure_times = (j < m_cov+1) ? when_to_leave(j) : [prev_depart_time+travel_time+1]
            
            # now we have to consider ALL possible departure times.
            for dt in departure_times

                if dt <= new_spot_arrival # if you depart when you arrive, what's the point?
                    continue
                end
                #only viable departure times remain
                
                # only visit non-depot if there are still jobs left
                if j != m_cov + 1
                    if dt > last_job_ended[j]
                        continue # it's possible to add zero routes from this, use depot?
                    end
                end
                
                cur_rc = rcs[curp]
                cur_rc += dists[last(curpath)+1, j+1]

                if j != m_cov + 1
                    for t in new_spot_arrival:dt
                        for i in neighborhood(j)
                            cur_rc -= xi[i, Int(t)]
                        end
                    end
                end
                
                if j < m_cov + 1 && cur_rc < pruner[j]
                    pruner[j] = cur_rc
                elseif j == m_cov+1
                else
                    continue
                end

                push!(paths, push!(copy(curpath), j))
                push!(rcs, cur_rc)
                if j < m_cov + 1
                    push!(times, push!(push!(copy(curtimes), new_spot_arrival), dt))
                else
                    push!(times, push!(copy(curtimes), new_spot_arrival))
                end

                endp += 1
            end
        end
        
        curp += 1
        
        if curp == endp
            break
        end
    end
    
    return paths, rcs, times
end;

# [9] Restricted Master Problem

In [1557]:
L = compute_L(cov_locs, job_locs);

In [1558]:
model = Model(Gurobi.Optimizer);
set_optimizer_attribute(model, "OutputFlag", 0);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [1559]:
last_z = 0
last_x = 0
rcs_q = []
rcs_p = []
last_objective = 0;

In [1560]:
keep_going = true
while keep_going
    C_jobs = [route_cost(route) for route in job_routes];
    C_covs = [route_cost(route) for route in cov_routes];
    u = compute_u(job_routes);
    delta = compute_delta(job_routes);
    y = compute_y(cov_routes);
    Q = length(job_routes)
    P = length(cov_routes);
    
    unregister(model, :z)
    @variable(model, 0 <= z[1:Q] <= 1);
    unregister(model, :x)
    @variable(model, 0 <= x[1:P] <= 1);
    
    unregister(model, :job_visited_once)
    @constraint(model, job_visited_once[i in 1:n_jobs], sum(u[i][q] * z[q] for q in 1:Q) == 1);
    unregister(model, :enough_job_vehicles)
    @constraint(model, enough_job_vehicles, sum(z[q] for q in 1:Q) <= B);
    unregister(model, :enough_cov_vehicles)
    @constraint(model, enough_cov_vehicles, sum(x[p] for p in 1:P) <= V);
    unregister(model, :must_be_covered)
    @constraint(model, must_be_covered[i in 1:n_jobs, t in 1:T], 
    sum(sum(y[j][t][p] * x[p] for j in 1:m_cov) for p in close_covs(i)) >= 
    sum(z[q] * delta[i][t][q] for q in 1:Q));
    
    @objective(model, Min, 
        sum(C_jobs[q] * z[q] for q in 1:Q) + 
        sum(C_covs[p] * x[p] for p in 1:P));
    
    optimize!(model)
    # println(objective_value(model))
    last_z = value.(z)
    last_x = value.(x)
    last_objective = objective_value(model)
    
    pi = dual.(job_visited_once)
    rho = dual.(enough_job_vehicles)
    beta = dual.(enough_cov_vehicles)
    xi = dual.(must_be_covered);
    
    contender_job_paths, contender_job_rcs, contender_job_times = job_sp(job_dists, job_travel_times, time_windows, work_load, rho, pi, xi);
    contender_cov_paths, contender_cov_rcs, contender_cov_times = cov_sp(cov_dists, cov_travel_times, beta, xi)
    
    viable_job_paths, viable_job_rcs, viable_job_times = [], [], []
    for index in 1:length(contender_job_paths)
        contender_path = contender_job_paths[index]
        contender_rc = contender_job_rcs[index]
        if last(contender_path) == n_jobs + 1 && contender_rc < -1e-8
            push!(viable_job_paths, contender_path)
            push!(viable_job_rcs, contender_rc)
            push!(viable_job_times, contender_job_times[index])
        end
    end
    
    viable_cov_paths, viable_cov_rcs, viable_cov_times = [], [], []
    for index in 1:length(contender_cov_paths)
        contender_path = contender_cov_paths[index]
        contender_rc = contender_cov_rcs[index]
        if last(contender_path) == n_jobs + 1 && contender_rc < -1e-8
            push!(viable_cov_paths, contender_path)
            push!(viable_cov_rcs, contender_rc)
            push!(viable_cov_times, contender_cov_times[index])
        end
    end
    
    if length(viable_job_paths) < 1 && length(viable_cov_paths) < 1
        println("ENDED!")
        keep_going = false
        break
    end
    
    mindex = -1
    best_rc = Inf

    for i in 1:length(viable_job_paths)
        if viable_job_rcs[i] < best_rc
            best_rc = viable_job_rcs[i]
            mindex = i
        end
    end
    
    push!(rcs_q, best_rc)
    
    # println(best_rc)
    
    viable_job_route = nodes_and_times_to_route(viable_job_paths[mindex], viable_job_times[mindex], "job");
    push!(job_routes, viable_job_route);
    
    mindex = -1
    best_rc = Inf
    for i in 1:length(viable_cov_paths)
        if viable_cov_rcs[i] < best_rc
            best_rc = viable_cov_rcs[i]
            mindex = i
        end
    end
    
    push!(rcs_p, best_rc)
    
    viable_cov_route = nodes_and_times_to_route(viable_cov_paths[mindex], viable_cov_times[mindex], "cov")
    push!(cov_routes, viable_cov_route);
    
    #=
    if viable_route == job_routes[end-1]
        println("Same route encountered")
        break
    end
    =#
end

LoadError: InterruptException:

In [1561]:
rcs_q

27-element Vector{Any}:
 -2922.616640431099
 -2259.2263063935006
 -2355.8463509136454
 -2726.6911370744224
 -1878.7618430150942
 -1819.6675718575577
 -2197.879655712386
 -1962.4702186520979
 -2194.3162462819273
 -1675.052584982577
 -2050.444471915099
 -1908.7578178143947
 -1639.879842770253
     ⋮
 -1275.6477732544165
 -1431.6619664416257
 -1318.0407145875834
 -1439.3151306528416
 -1367.6623360278875
 -1187.8953496540496
 -1187.8953496540496
 -1187.8953496540496
 -1187.8953496540496
 -1187.8953496540496
 -1187.8953496540496
 -1187.8953496540496

In [1562]:
rcs_p

27-element Vector{Any}:
  -578.5905068916492
  -735.9824431621989
  -565.1187169058121
  -565.1187169058121
  -439.56647210208894
 -1023.7095852635373
  -319.76444956307057
  -508.41198636634664
  -768.4267349657708
  -553.8727306558897
 -1172.3722476619455
 -1087.2690550515947
  -895.9210280561847
     ⋮
  -872.3991835489763
  -747.6616084261877
  -846.477755428028
 -1285.4966192675208
 -1550.7646934856846
 -1286.747816774688
 -1286.747816774688
 -1286.747816774688
 -1286.747816774688
 -1286.747816774688
 -1286.747816774688
 -1286.747816774688